In [ ]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from colorama import Fore
from urllib.parse import urlparse
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from tld import get_tld, is_tld
from sklearn import model_selection
from sklearn.ensemble import VotingClassifier
import import_ipynb
import features as f
#from ML_implementation_Algorithm import ml_system
from features import abnormal_url
from features import httpSecure
from features import digit_count
from features import letter_count
from features import Shortining_Service
from features import having_ip_address
from flask import Flask, request, url_for, render_template
import import_ipynb


app = Flask(__name__)

@app.route('/')
def home():
    return render_template('home.html')


@app.route('/predict', methods= ["POST"])
def ml_system():
    
    url_link = str(request.form['name1'].lower())
    data = pd.DataFrame([url_link], columns=["url"])
    
    data["url"] = data['url'].replace('www.', '', regex=True)
    data['url_len'] = data['url'].apply(lambda x: len(str(x)))
    feature = ['@','?','-','=','.','#','%','+','$','!','*',',','//']
    for a in feature:
        data[a] = data['url'].apply(lambda i: i.count(a))
    data['abnormal_url'] = data['url'].apply(lambda i: abnormal_url(i))
    data['https'] = data['url'].apply(lambda i: httpSecure(i))
    data['digits']= data['url'].apply(lambda i: digit_count(i))
    data['letters']= data['url'].apply(lambda i: letter_count(i))
    data['Shortining_Service'] = data['url'].apply(lambda x: Shortining_Service(x))
    data['having_ip_address'] = data['url'].apply(lambda i: having_ip_address(i))


    df_new = pd.read_csv("comprehensive_phishing_data.csv")
    df_new = df_new.sample(frac = 0.05)
    X = df_new.drop(['url', 'target'],axis=1)#,'type_code'
    y = df_new['target']

            ##split dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

            ###Ensemble learning 
    kfold = model_selection.KFold(n_splits=10)
    model1 = DecisionTreeClassifier(random_state =0)
    model2 = RandomForestClassifier(n_estimators=10)
    model3 = KNeighborsClassifier()
    model4 = GaussianNB()
            # create the sub models
    estimators = []
    estimators.append(('Decision_Tree', model1))
    estimators.append(('Random_Forest', model2))
    estimators.append(('KNN', model3))
    estimators.append(('NB', model4))
            # create the ensemble model
    ensemble = VotingClassifier(estimators)
            ##fit data
    ensemble.fit(X_train,y_train)

    arr_data = []
    for n in data.values:
        n_values = n[1:]
        for i in n_values:
            arr_data.append(i)
    pred_value = ensemble.predict([arr_data])
    for k in pred_value:
        if k==0:
            out_put = f"The Url link is Benign, continue to site: {url_link}"
        else:
            out_put = "The link seems like a Phishing link"             
    
    return render_template('home.html', output_value=out_put)


if __name__ == '__main__':
    app.run(host="localhost", port=8000, debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://localhost:8000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [07/Sep/2022 22:15:39] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Sep/2022 22:15:40] "GET /static/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Sep/2022 22:15:40] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [07/Sep/2022 22:15:52] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [07/Sep/2022 22:15:52] "GET /static/style.css HTTP/1.1" 304 -
